In [9]:
"""
Defining all the necessary dictionaries for Escape Room objects.

Listed dictionaries include:
    1. Couch = couch
    2. Bed = bed
    3. Piano = piano
    4. Dining Table = dining_table
    7. Keys = key_A, key_C, key_D
    8. Rooms = game_room, bedroom_1, bedroom_2, living_room, outside
"""

# Creating dictionaries
couch = {'name': 'Couch', 
         'type': 'furniture'}

piano = {'name': 'Piano',
         'type': 'instrument'}
game_room = {"name": "Game Room",
             "type": "room"}
key_A = {'name': 'Key A',
         'type': 'key',
         'target': 'Door A'}

# Bedroom 2 
bedroom_2 = {'name': 'Bedroom 2',
             'type': 'room' }
double_bed = {'name': 'Double Bed',
          'type': 'furniture'}
dresser = {'name': 'Dresser',
           'type': 'furniture'}
key_C = {'name': 'Key C',
         'type': 'key',
         'target': 'Door C'}
key_D = {'name': 'Key D',
         'type': 'key',
         'target': 'Door D'}

#Living Room 

living_room = {'name': 'Living Room',
             'type': 'room' }  
dining_table = {'name': 'Dining Table',
       'type': 'furniture'}
# Bedroom 1
bedroom_1 ={'name': 'Bedroom 1',
            'type': 'room'}
queen_bed = {'name': 'Queen Bed',
          'type': 'furniture'}
key_B = {'name': 'Key B',
         'type': 'key',
         'target': 'Door B'}
# Outside
outside = {'name': 'Outside'}

#Doors 

door_A = {'name': 'Door A',
          'type': 'door',
          'locked': True,
          'key': 'key_A'}
door_B= {'name': 'Door B',
          'type': 'door',
          'locked': True,
          'key': 'key_B'}
door_C = {'name': 'Door C',
          'type': 'door',
          'locked': True,
          'key': 'key_C'}
door_D = {'name': 'Door D',
          'type': 'door',
          'locked': True,
          'key': 'key_D'}


"""
since we define all the dictionaries

"""
#defining all room objects
all_rooms = ['game_room', 'bedroom_1', 'bedroom_2', 'living_room', 'outside']
all_doors= ['door_A', 'door_B','door_C', 'door_D']
# define which items/rooms are related

# Defining the relashion between the object and rooms
object_relations = {
    'game_room': [couch,piano,door_A],
    'piano' :  [key_A],
    'door_A': [game_room,bedroom_1],
    'bedroom_2': [double_bed,dresser,door_B],
    'double_bed': [key_C],
    'dresser': [key_D],
    'door_B': [bedroom_1,bedroom_2],
    'living_room': [dining_table,door_C,door_D]
    
}
# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [10]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if(next_room and input("Do you want to go to the next room? Ener 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [12]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in Game Room


KeyError: 'Game Room'